### Model Training

In [1]:
# from ultralytics import YOLO

# model = YOLO("yolov8n.pt")  

# model.train(
#     data="Dataset/data.yaml",  
#     epochs=50,           
#     imgsz=640,            
#     batch=16,             
#     device="cuda"   
# )

# model_path = "runs/detect/train/weights/best.pt"

### Detection + OCR

In [32]:
import os
import glob
import cv2
from ultralytics import YOLO

model = YOLO("runs/detect/train5/weights/best.pt")

input_dir = "D:/Freelancing/Number Plate/OCR Dataset/test/"
output_dir = "D:/Freelancing/Number Plate/OCR Dataset/test_cropped/"

os.makedirs(output_dir, exist_ok=True)

image_paths = glob.glob(os.path.join(input_dir, "*.jpg"))

for image_path in image_paths:
    image = cv2.imread(image_path)

    results = model(image)

    for i, result in enumerate(results):
        for j, box in enumerate(result.boxes.xyxy):
            x_min, y_min, x_max, y_max = map(int, box[:4].tolist())

            padding = 10  # Adjust the padding value as needed
            x_min_padded = max(0, x_min - padding)
            y_min_padded = max(0, y_min - padding)
            x_max_padded = min(image.shape[1], x_max + padding)
            y_max_padded = min(image.shape[0], y_max + padding)
            cropped_plate = image[y_min_padded:y_max_padded, x_min_padded:x_max_padded]

            output_path = os.path.join(output_dir, f"cropped_plate_{os.path.basename(image_path).split('.')[0]}_{i}_{j}.jpg")
            cv2.imwrite(output_path, cropped_plate)

print("License plate(s) successfully detected and saved.")


0: 640x640 1 license-plate, 26.2ms
Speed: 5.9ms preprocess, 26.2ms inference, 4.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 license-plate, 20.7ms
Speed: 5.5ms preprocess, 20.7ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 license-plate, 20.8ms
Speed: 4.4ms preprocess, 20.8ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 license-plate, 20.5ms
Speed: 3.4ms preprocess, 20.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 license-plate, 20.0ms
Speed: 5.1ms preprocess, 20.0ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 license-plate, 18.4ms
Speed: 4.5ms preprocess, 18.4ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 license-plate, 20.6ms
Speed: 2.0ms preprocess, 20.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 license-plate, 18.7ms
Speed: 6.2ms preprocess, 18.7ms i

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
from skimage import measure
from skimage.filters import threshold_local
from skimage.segmentation import clear_border
import imutils
import os

# Load character templates from a directory
def load_templates(template_dir):
    templates = {}
    for filename in os.listdir(template_dir):
        if filename.endswith('.png'):
            char_label = os.path.splitext(filename)[0]
            template_path = os.path.join(template_dir, filename)
            template_image = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
            templates[char_label] = template_image
    return templates

# Detect license plate using YOLO
def detect_license_plate(image, model):
    results = model(image)
    for result in results:
        for box in result.boxes.xyxy:
            x_min, y_min, x_max, y_max = map(int, box[:4].tolist())  # Ensure correct indexing
            cropped_plate = image[y_min:y_max, x_min:x_max]
            return cropped_plate
    return None

# Convert image to grayscale
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray

# Segment characters from a license plate image
def segment_characters(plate_image):
    gray = preprocess_image(plate_image)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 45, 15)
    _, labels = cv2.connectedComponents(thresh)
    mask = np.zeros(thresh.shape, dtype="uint8")
    total_pixels = plate_image.shape[0] * plate_image.shape[1]
    lower = total_pixels // 70
    upper = total_pixels // 20

    for label in np.unique(labels):
        if label == 0:
            continue
        label_mask = np.zeros(thresh.shape, dtype="uint8")
        label_mask[labels == label] = 255
        numPixels = cv2.countNonZero(label_mask)
        if lower < numPixels < upper:
            mask = cv2.add(mask, label_mask)

    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = [cv2.boundingRect(c) for c in contours]
    bounding_boxes = sorted(bounding_boxes, key=lambda b: (b[1], b[0]))

    char_images = [mask[y:y+h, x:x+w] for (x, y, w, h) in bounding_boxes]
    return char_images

# Template matching using NCC & Phase Correlation
def match_template(char_image, templates):
    best_match_ncc, best_score_ncc = None, -1
    best_match_phase, best_score_phase = None, -1
    resized_char = cv2.resize(char_image, (50, 50))

    for label, template in templates.items():
        template_resized = cv2.resize(template, (50, 50))

        # Normalized Cross-Correlation (NCC)
        result = cv2.matchTemplate(resized_char, template_resized, cv2.TM_CCOEFF_NORMED)
        score_ncc = result.max()
        if score_ncc > best_score_ncc:
            best_score_ncc = score_ncc
            best_match_ncc = label

        # Phase Correlation
        shift, response = cv2.phaseCorrelate(np.float32(resized_char), np.float32(template_resized))
        if response > best_score_phase:
            best_score_phase = response
            best_match_phase = label

    # print(f"Best match (NCC): {best_match_ncc} ({best_score_ncc})")
    # print(f"Best match (Phase Correlation): {best_match_phase} ({best_score_phase})")
    return best_match_ncc, best_score_ncc, best_match_phase, best_score_phase

# Recognize characters from segmented images
def recognize_characters(segmented_chars, templates):
    recognized_text_ncc = ""
    recognized_text_phase = ""

    for char_img in segmented_chars:
        match_ncc, score_ncc, match_phase, score_phase = match_template(char_img, templates)
        if match_ncc:
            recognized_text_ncc += match_ncc
        if match_phase:
            recognized_text_phase += match_phase

    return recognized_text_ncc, recognized_text_phase

# Main function to run the full pipeline
def main(image_path, model_path, template_dir):
    model = YOLO(model_path)
    templates = load_templates(template_dir)
    image = cv2.imread(image_path)

    results = model(image)
    plate_count = 0

    for result in results:
        for box in result.boxes.xyxy:
            x_min, y_min, x_max, y_max = map(int, box[:4].tolist())  # Ensure correct indexing
            cropped_plate = image[y_min:y_max, x_min:x_max]

            plate_filename = f"cropped_plate_{plate_count}.jpg"
            cv2.imwrite(plate_filename, cropped_plate)
            plate_count += 1

            segmented_chars = segment_characters(cropped_plate)
            print(f"Segmented {len(segmented_chars)} characters")
            for i, char_img in enumerate(segmented_chars):
                char_filename = f"segmented_char_{plate_count}_{i}.jpg"
                cv2.imwrite(char_filename, char_img)

            recognized_text_ncc, recognized_text_phase = recognize_characters(segmented_chars, templates)

            print(f"Recognized License Plate (NCC): {recognized_text_ncc}")
            print(f"Recognized License Plate (Phase Correlation): {recognized_text_phase}")

In [5]:
image_path = "D:/Freelancing/Number Plate/Dataset/Original/18.jpg"  
model_path = "runs/detect/train4/weights/best.pt"  
template_dir = "D:/Freelancing/Number Plate/Templates/CharacterImages"  
main(image_path, model_path, template_dir)


0: 480x640 1 license-plate, 44.2ms
Speed: 4.2ms preprocess, 44.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Segmented 2 characters
Recognized License Plate (NCC): X12
Recognized License Plate (Phase Correlation): W1X1


In [25]:
from PIL import Image
import pytesseract

# Configure tesseract executable path (update this path if necessary)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def recognize_text(image_path):
    """Use Tesseract OCR to recognize text from the image."""
    try:
        # Open the image file
        with Image.open(image_path) as img:
            # Use Tesseract to do OCR on the image
            text = pytesseract.image_to_string(img)
            print(f"Recognized text: {text}")
            return text.strip()
    except Exception as e:
        print(f"Error: {e}")
        return None

In [27]:
# image_directory = 'Dataset/Cropped/'
# for filename in os.listdir(image_directory):
#     if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
#         print(f"\nProcessing {filename}:")
#         recognize_text(os.path.join(image_directory, filename))

In [30]:
from fast_plate_ocr import ONNXPlateRecognizer

m = ONNXPlateRecognizer('argentinian-plates-cnn-model')
print(m.run('Dataset\Cropped\cropped_plate_112_0_0.jpg'))

*************** EP Error ***************
EP Error D:\a\_work\1\s\onnxruntime\python\onnxruntime_pybind_state.cc:507 onnxruntime::python::RegisterTensorRTPluginsAsCustomOps Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************
['A0551TT']
